In [ ]:
import pandas as pd
import time
import os
import sys
from Detectors.model_wrapper import run_Unsupervise_AD, run_Semisupervise_AD
from Detectors.evaluation.metrics import get_metrics
from filelock import FileLock
from datasets import load_dataset

# Load the CalIt2 dataset
data_name = "Calit2"
df = pd.read_csv("Datasets/mTSBench/CalIt2/CalIt2_traffic_test.csv")

# using KMeansAD
detector = 'KMeansAD'

data = df.iloc[:, 1:-1].values.astype(float) # excluede first (time stamp) and last (label) columns 
label = df['is_anomaly'].astype(int).to_numpy()

start = time.time()
output = run_Unsupervise_AD(detector, data) # change this to run_Semisupervise_AD if the detector is semi-supervised
end = time.time()
runtime = end - start

metrics = get_metrics(output, label)
record = {
    "data_file": data_name, 
    "model": detector,
    "runtime": runtime,
    **metrics
}

# Save results to results/<detector>_evaluation_results.csv
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

log_file = os.path.join(results_dir, f"{detector}_evaluation_results.csv")
with FileLock(log_file + ".lock"):
    df_log = pd.DataFrame([record])
    write_header = not os.path.exists(log_file)
    df_log.to_csv(log_file, mode='a', header=write_header, index=False)